In [53]:
import os
from crewai import Agent, Crew, Process, Task ,LLM

In [54]:
from crewai_tools import SerperDevTool ,ScrapeWebsiteTool

In [55]:
model='azure/gpt-4.1'
AZURE_API_KEY='C0AobTHxvT16LiGF5WN9x3nBgFcQK3B8NtcTVJLKyGMDlyZfzF0FJQQJ99BDACHYHv6XJ3w3AAAAACOGVWPp'
AZURE_API_BASE='https://azai-useast2-synergy.cognitiveservices.azure.com/'
AZURE_API_VERSION='2024-12-01-preview'
SERPER_API_KEY='d0540e84ba2200f8e2eb88bc81d3b3f489b523ff'

In [56]:
os.environ["AZURE_API_KEY"] = AZURE_API_KEY
os.environ["AZURE_API_BASE"] = AZURE_API_BASE
os.environ["AZURE_API_VERSION"] = AZURE_API_VERSION
os.environ["model"] = model


In [57]:
serper_api_key='d0540e84ba2200f8e2eb88bc81d3b3f489b523ff'
if not os.environ.get("SERPER_API_KEY"):
    os.environ["SERPER_API_KEY"] = serper_api_key

In [58]:
lead_researcher = Agent(
    role="Company Researcher",
    goal="Gather comprehensive information about the target company",
    tools=[SerperDevTool(),ScrapeWebsiteTool()],
    backstory= "You are an expert at researching companies, with a keen eye for details that matter to sales professionals. Your task is to gather key information about {target_company} that will be relevant for identifying potential buyers of {our_product}.",
    verbose=True,
    allow_delegation=False,
    llm='azure/gpt-4.1'
)

In [59]:
contact_finder = Agent(
    role="Contact_finder",
    goal="Find specific individuals in key roles at the target company",
    tools=[SerperDevTool(),ScrapeWebsiteTool()],
    backstory="You are skilled at identifying and locating specific individuals within organizations. Your task is to find the names, titles, and contact information of potential decision-makers at {target_company} who might be interested in {our_product}.",
    verbose=True,
    allow_delegation=False,
    llm='azure/gpt-4.1'
)

In [60]:
org_structure_analyst = Agent(
    role="Organizational Structure Analyst",
    goal="Analyze the company's structure to identify key decision-making roles",
    tools=[SerperDevTool(),ScrapeWebsiteTool()],
    backstory="You are skilled at identifying and locating specific individuals within organizations. Your task is to find the names, titles, and contact information of potential decision-makers at {target_company} who might be interested in {our_product}.",
    verbose=True,
    allow_delegation=False,
    llm='azure/gpt-4.1'
)

In [61]:
sales_strategist = Agent(
    role="Sales Approach Strategist",
    goal="Develop a strategy for approaching the identified contacts inline with Synergy's Offerings and Strategic goals",
    tools=[SerperDevTool(),ScrapeWebsiteTool(website_url='https://www.synergymarinegroup.com'),ScrapeWebsiteTool(website_url='https://www.synergymarinegroup.com/services/technical-management/')],
    backstory="You work for Synergy Maritime and have complete understanding of our offerings from the Websearch tool provided to you.You also have context of the tasks that agents before you have complete - ensure you utilize these in framing your final answer. You are an experienced sales professional with a talent for developing effective outreach strategies.Your role is to create an approach plan for contacting the identified individuals at {target_company} about {our_product}.",
    verbose=True,
    allow_delegation=False,
    llm='azure/gpt-4.1'
)

In [62]:
research_company_task = Task(
    description="Conduct thorough research on {target_company}. Focus on their industry,size, recent news, challenges they might be facing, and any information that suggests they might benefit from {our_product}. Look for recent technology initiatives or digital transformation efforts.",
    expected_output="A comprehensive overview of {target_company}, including key facts,recent developments, and potential pain points that our product could address.",
    agent=lead_researcher,
    async_execution=False,
    llm='azure/gpt-4.1'
)

In [63]:
analyze_org_structure_task = Task(
    description="Analyze the organizational structure of {target_company}. Identify key departments and roles that are likely to be involved in decisions related to purchasing {our_product}. Consider both technical decision-makers and business stakeholders.",
	expected_output="A breakdown of {target_company}'s relevant organizational structure, highlighting departments and roles that are potential stakeholders for {our_product}.",
    agent=sales_strategist,
    async_execution=False,
    llm='azure/gpt-4.1'
)     

In [64]:
find_key_contacts_task = Task(
    description="Based on the organizational analysis, identify specific individuals at {target_company} who are in positions to influence or make decisions about purchasing {our_product}.Find their names, titles, and if possible, professional contact information or LinkedIn profiles. Ensure we have atleast 5-6 contacts across various levels in the organization",
    expected_output="A list of key contacts at {target_company}, including their names, titles, departments, and any available contact information, LinkedIn URLs if possible phones and emails." \
    "Format for each of the contacts gathered " \
    "Name :   Role :  LinkedIN URL:  Phone : Email:" \
    "Name :   Role :  LinkedIN URL:  Phone : Email:" \
    "Name :   Role :  LinkedIN URL:  Phone : Email:",
    agent=contact_finder,
    async_execution=False,
    llm='azure/gpt-4.1',
)

In [65]:
develop_approach_strategy_task = Task(
    description="Using all the gathered information about {target_company} and the identified contacts, develop a strategy for approaching these individuals about {our_product}. Consider the company's challenges, the contacts' roles, and how our product can provide value to them.",
    expected_output=
        "A JSON object with the following keys:\n"
        "- company_overview: A JSON object with:\n"
        "  - name: String, the company’s name from [search_company_task].\n"
        "  - headquarters: String, the company’s headquarters location.\n"
        "  - core_business: String, the company’s primary business activities.\n"
        "  - size: String, the company’s scale (e.g., employee count, operational scope).\n"
        "  - leadership: String, key leadership roles and names.\n"
        "  - recent_initiatives: Array of strings, recent projects or initiatives.\n"
        "  - strategic_goals: Array of strings, the company’s strategic objectives.\n"
        "  - overlay_opportunities: Array of strings, opportunities where {target_company}’s initiatives align with {our_product}’s objectives.\n"
        "- key_contacts: An array of JSON objects from [find_key_contacts_task], each with:\n"
        "  - name: String, contact’s full name.\n"
        "  - role: String, contact’s job title or role.\n"
        "  - LinkedIn URL: String or null, contact’s LinkedIn profile URL.\n"
        "  - phone: String, contact’s phone number or general company contact if unavailable.\n"
        "  - email: String, contact’s email or general company email if unavailable.\n"
        "- tailored_approach_strategy: A JSON object with:\n"
        "  - general_principles: Array of strings, high-level guidelines for outreach (e.g., personalization, value proposition, channels).\n"
        "  - contact-specific_outreach_methods: JSON object, keys as contact names (e.g., 'Name (Role)') and values as strings describing tailored outreach methods.\n"
        "  - sample_messaging: JSON object, keys as messaging categories (e.g., 'for_CEO_and_Exec_Board', 'for_Fleet/Tech/Innovation_Leads', 'for_HR/Crew_Management') and values as strings with sample outreach messages.\n"
        "  - rationale: Array of strings, explanations of why the approach suits the contacts’ priorities and how {our_product} adds value."
    ,
    agent=sales_strategist,
    async_execution=False,
    context=[research_company_task,find_key_contacts_task,analyze_org_structure_task],

    llm='azure/gpt-4.1',

)

In [66]:


target_company= 'Sea1 Offshore'#'Tidewater Inc.'#'DOF Group'#'Louis Dreyfus Armateurs'#'petership'
our_product = 'Ship Management'
# Build and Run Crew
inputs = {
        "target_company": target_company,
        "our_product": our_product,
    }

# Build and Run Crew
crew = Crew(
    agents=[lead_researcher, org_structure_analyst, contact_finder, sales_strategist],
    tasks=[research_company_task, analyze_org_structure_task, find_key_contacts_task, develop_approach_strategy_task],
	process=Process.sequential,
	manager_llm='azure/gpt-4.1',
	verbose=True,	
)

result = crew.kickoff(inputs=inputs);
#TaskOutput = result.tasks[0].output
# If using a newer version (e.g., >=0.51.x)
#try:
#    task_output = result.tasks_output[-1]  # Get the TaskOutput for develop_approach_strategy_task
#    print("TaskOutput attributes:", dir(task_output))  # Inspect available attributes
#except Exception as e:
#    print(f"Error accessing TaskOutput: {e}")
#if hasattr(result, 'tasks_output'):
    # Access the output of the last task (develop_approach_strategy_task)
#    print("Tasks Output:")
#    crew_output = result.tasks_output[-1].raw
#    print(crew_output)
if hasattr(result, 'raw'):
    # Raw output as a string or JSON
    print("Raw Output:")
    crew_output = result.raw
    print(crew_output)
else:
    raise AttributeError("Unable to find valid output attribute in CrewOutput")

#print("\nFinal Output:\n", CrewOutput)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d86ce028-fb92-48a1-a125-e22ebb2badcf                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 974f6b22-e12d-4287-b02e-cbfb8560066d
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 974f6b22-e12d-4287-b02e-cbfb8560066d
       Status: Executing Task...
    └── 🤖 Agent: Company Researcher
            Status: In Progress

# Agent: Company Researcher
## Task: Conduct thorough research on Sea1 Offshore. Focus on their industry,size, recent news, challenges they might be facing, and any information that suggests they might benefit from Ship Management. Look for recent technology initiatives or digital transformation efforts.


🤖 Agent: Company Researcher
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 974f6b22-e12d-4287-b02e-cbfb8560066d
│      Assigned to: Company Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Company Researcher
│           Status: ✅ Completed
├── 📋 Task: 245307ca-cde4-4686-8dbf-38d8aef328f9
│      Assigned to: Sales Approach Strategist
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Approach Strategist
│           Status: ✅ Completed
├── 📋 Task: 94200a81-b623-45d0-9261-98dcf60b39e6
│      Assigned to: Contact_finder
│      Status: ✅ Completed
│   └── 🤖 Agent: Contact_finder
│           Status: ✅ Completed
└── 📋 Task: c3e8ae9c-7495-41e4-abec-fd320d7ca364
       Status: Executing Task...
    └── 🤖 Agent: Sales Approach Strategist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 974f6b22-e12d-4287-b02e-cbfb8560066d
│      Assigned to: Company Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Company Researcher
│           Status: ✅ Completed
├── 📋 Task: 245307ca-cde4-4686-8dbf-38d8aef328f9
│      Assigned to: Sales Approach Strategist
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Approach Strategist
│           Status: ✅ Completed
├── 📋 Task: 94200a81-b623-45d0-9261-98dcf60b39e6
│      Assigned to: Contact_finder
│      Status: ✅ Completed
│   └── 🤖 Agent: Contact_finder
│           Status: ✅ Completed
└── 📋 Task: c3e8ae9c-7495-41e4-abec-fd320d7ca364
       Assigned to: Sales Approach Strategist
       Status: ✅ Completed
    └── 🤖 Agent: Sales Approach Strategist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c3e8ae9c-7495-41e4-abec-fd320d7ca364                                                                     │
│  Agent: Sales Approach Strategist                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d86ce028-fb92-48a1-a125-e22ebb2badcf                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Raw Output:
```json
{
  "company_overview": {
    "name": "Sea1 Offshore",
    "headquarters": "Norway",
    "core_business": "Provider of vessels for offshore oil & gas and renewable industries, offering a fleet for offshore support, supply, subsea construction, well intervention, and scientific drilling operations.",
    "size": "Approx. 1200 employees (offshore/onshore), 34 vessels, international presence in Norway, Canada, Australia, USA, Brazil; listed on Oslo Børs; growing international operations.",
    "leadership": "Bernt Omdal (CEO), Vidar Jerstad (CFO), Tore Lillestø (COO), Andreas Kjøl (CCO), Tor Asbjørn Grændsen (CHRO). Board: Christen Sveaas (Chairman), Celina Midelfart, Fredrik Baksaas, Hans Petter Aas.",
    "recent_initiatives": [
      "Rebrand from Siem Offshore to Sea1 Offshore",
      "Acquisition and newbuilds of vessels, sale of 9 vessels to major shareholder (April 2024)",
      "New international contracts and vessel extensions",
      "Collaboration with Wärts

In [67]:
crew_output

'```json\n{\n  "company_overview": {\n    "name": "Sea1 Offshore",\n    "headquarters": "Norway",\n    "core_business": "Provider of vessels for offshore oil & gas and renewable industries, offering a fleet for offshore support, supply, subsea construction, well intervention, and scientific drilling operations.",\n    "size": "Approx. 1200 employees (offshore/onshore), 34 vessels, international presence in Norway, Canada, Australia, USA, Brazil; listed on Oslo Børs; growing international operations.",\n    "leadership": "Bernt Omdal (CEO), Vidar Jerstad (CFO), Tore Lillestø (COO), Andreas Kjøl (CCO), Tor Asbjørn Grændsen (CHRO). Board: Christen Sveaas (Chairman), Celina Midelfart, Fredrik Baksaas, Hans Petter Aas.",\n    "recent_initiatives": [\n      "Rebrand from Siem Offshore to Sea1 Offshore",\n      "Acquisition and newbuilds of vessels, sale of 9 vessels to major shareholder (April 2024)",\n      "New international contracts and vessel extensions",\n      "Collaboration with Wärt

In [68]:
import json

def extract_json_text_from_markdown(text):
    """
    Extracts JSON content from between ```json ``` markers in a text string.
    
    Args:
        text (str): The input text containing JSON between markers
        
    Returns:
        dict/list: The parsed JSON content
    """
    # Define the start and end markers
    start_marker = "```json"
    end_marker = "```"
    
    # Find the positions of the markers
    start_pos = text.find(start_marker)
    if start_pos == -1:
        raise ValueError("Start marker ```json not found in text")
    
    end_pos = text.find(end_marker, start_pos + len(start_marker))
    if end_pos == -1:
        raise ValueError("End marker ``` not found in text")
    
    # Extract the JSON content
    json_content = text[start_pos + len(start_marker):end_pos].strip()
    
    # Parse and return the JSON
    return json_content


In [69]:


def transform_raw_data_to_json(raw_string):
	"""
	Transforms raw string data (with markdown code block) into a proper JSON object

	Args:
		raw_string (str): The raw string containing markdown code block with JSON
		
	Returns:
		dict: The parsed JSON data as a Python dictionary
	"""
	# Remove the markdown code block markers
	if raw_string.startswith('```json') and raw_string.endswith('```'):
		json_str = raw_string[7:-3].strip()  # Remove first 7 and last 3 chars
	else:
		if raw_string.startswith('```json'):
			json_str = extract_json_text_from_markdown(raw_string)
		else:
			json_str = raw_string.strip()  # If no markers, just strip whitespace
		#json_str = raw_string[7:-3].strip()  # Remove first 7 and last 3 chars
	# Parse the JSON string into a Python dictionary
	try:
		data = json.loads(json_str)
		return data
	except json.JSONDecodeError as e:
		print(f"Error parsing JSON: {e}")
		return None

# Transform the data
json_data = transform_raw_data_to_json(crew_output)
print(json_data)
# Optionally, save to a file

# The json_data variable now contains the properly formatted Python dictionary

{'company_overview': {'name': 'Sea1 Offshore', 'headquarters': 'Norway', 'core_business': 'Provider of vessels for offshore oil & gas and renewable industries, offering a fleet for offshore support, supply, subsea construction, well intervention, and scientific drilling operations.', 'size': 'Approx. 1200 employees (offshore/onshore), 34 vessels, international presence in Norway, Canada, Australia, USA, Brazil; listed on Oslo Børs; growing international operations.', 'leadership': 'Bernt Omdal (CEO), Vidar Jerstad (CFO), Tore Lillestø (COO), Andreas Kjøl (CCO), Tor Asbjørn Grændsen (CHRO). Board: Christen Sveaas (Chairman), Celina Midelfart, Fredrik Baksaas, Hans Petter Aas.', 'recent_initiatives': ['Rebrand from Siem Offshore to Sea1 Offshore', 'Acquisition and newbuilds of vessels, sale of 9 vessels to major shareholder (April 2024)', 'New international contracts and vessel extensions', 'Collaboration with Wärtsilä for fleet decarbonisation', 'Installation/orders of battery packages 

In [82]:
country = json_data['company_overview']['headquarters']
country

'Norway'

In [70]:
json_data['key_contacts']


[{'name': 'Bernt Omdal',
  'role': 'Chief Executive Officer',
  'LinkedIn URL': 'https://no.linkedin.com/in/bernt-omdal-073985133',
  'phone': '+47 38 60 04 00',
  'email': 'info@sea1offshore.com'},
 {'name': 'Vidar Jerstad',
  'role': 'Chief Financial Officer',
  'LinkedIn URL': 'https://no.linkedin.com/in/vidar-jerstad-2b81874',
  'phone': '+47 38 60 04 00',
  'email': 'info@sea1offshore.com'},
 {'name': 'Tore Lillestø',
  'role': 'Chief Operating Officer',
  'LinkedIn URL': 'https://no.linkedin.com/in/tore-lillest%C3%B8-08b31b68',
  'phone': '+47 38 60 04 00',
  'email': 'info@sea1offshore.com'},
 {'name': 'Andreas Kjøl',
  'role': 'Chief Commercial Officer',
  'LinkedIn URL': 'https://no.linkedin.com/in/andreas-kj%C3%B8l-65678a37',
  'phone': '+47 38 60 04 00',
  'email': 'info@sea1offshore.com'},
 {'name': 'Tor Asbjørn Grændsen',
  'role': 'Chief Human Resources Officer',
  'LinkedIn URL': 'https://no.linkedin.com/in/tor-asbj%C3%B8rn-gr%C3%A6ndsen-9945064',
  'phone': '+47 38 60 0

In [72]:
key_contacts=json_data['key_contacts']

In [73]:
# Surfe Integration
SURFE_BEARER_TOKEN = "ctksSOUV1-VRSY8i-FNgZW7IhyjnrDZOVYAsnQvzDOA"
import requests


enrichment_ids = []
# Post Key Contacts to Surfe API for Enrichment
for contact in key_contacts:
    url = "https://api.surfe.com/v2/people/enrich"
    payload = {
        "include": {
            "email": True,
            "linkedInUrl": True,
            "mobile": True
        },
        "people": [
            {
                "companyDomain": "",
                "companyName": target_company,
                "externalID": "external-id",
                "firstName": contact['name'].split()[0] if 'name' in contact else '',
                "lastName": contact['name'].split()[-1] if 'name' in contact else '',
                "linkedinUrl": contact['LinkedIn URL'] if 'LinkedIn URL' in contact and contact['LinkedIn URL'] else ''
            }
        ]
    }
    
    headers = {
        "Authorization": f"Bearer {SURFE_BEARER_TOKEN}"
    }
    
    response = requests.request("POST", url, json=payload, headers=headers)
    print(f"Raw response: {response.text}")
    
    if response.status_code in [200, 201, 202]:
        try:
            response_data = response.json()  # Use response.json() instead of json.loads()
            enrichment_id = response_data.get('enrichmentID')
            enrichment_ids.append(enrichment_id)
            print(f"Enriched Contact ID: {enrichment_id}")
        except json.JSONDecodeError as e:
            print(f"Error parsing response for contact {contact['name']}: {e}")
    else:
        print(f"Failed to enrich contact {contact['name']}: {response.text}")

Raw response: {"message":"Your enrichment has started ✨, estimated time: 1s.","enrichmentID":"019734db-6b2e-7c3d-aada-27169d88d7e6","enrichmentCallbackURL":"https://api.surfe.com/v2/people/enrich/019734db-6b2e-7c3d-aada-27169d88d7e6"}

Enriched Contact ID: 019734db-6b2e-7c3d-aada-27169d88d7e6
Raw response: {"message":"Your enrichment has started ✨, estimated time: 1s.","enrichmentID":"019734db-92a5-7021-9985-1f4793bd62f0","enrichmentCallbackURL":"https://api.surfe.com/v2/people/enrich/019734db-92a5-7021-9985-1f4793bd62f0"}

Enriched Contact ID: 019734db-92a5-7021-9985-1f4793bd62f0
Raw response: {"message":"Your enrichment has started ✨, estimated time: 1s.","enrichmentID":"019734db-94db-78b7-bc6a-8c966895c289","enrichmentCallbackURL":"https://api.surfe.com/v2/people/enrich/019734db-94db-78b7-bc6a-8c966895c289"}

Enriched Contact ID: 019734db-94db-78b7-bc6a-8c966895c289
Raw response: {"message":"Your enrichment has started ✨, estimated time: 1s.","enrichmentID":"019734db-973a-7d0a-a858-

In [75]:
key_contacts_enriched_raw = []
#Enrich key contacts using Surfe API 
for enrichment_id in enrichment_ids:
    url = f"https://api.surfe.com/v2/people/enrich/{enrichment_id}"  # Add ID to URL
    headers = {
        "Authorization": f"Bearer {SURFE_BEARER_TOKEN}"
    }
    response = requests.request("GET", url, headers=headers)
    
    if response.status_code == 200:
        try:
            enrichment_data = response.json()
            key_contacts_enriched_raw.append(enrichment_data)
            print(f"Enrichment data for ID {enrichment_id}:")
            print(json.dumps(enrichment_data, indent=2))
        except json.JSONDecodeError as e:
            print(f"Error parsing response for ID {enrichment_id}: {e}")
    else:
        print(f"Error retrieving enrichment for ID {enrichment_id}: {response.text}")

Enrichment data for ID 019734db-6b2e-7c3d-aada-27169d88d7e6:
{
  "enrichmentID": "019734db-6b2e-7c3d-aada-27169d88d7e6",
  "status": "COMPLETED",
  "percentCompleted": 100,
  "people": [
    {
      "externalID": "external-id",
      "firstName": "Bernt",
      "lastName": "Omdal",
      "companyName": "Sea1 Offshore",
      "companyDomain": "sea1offshore.com",
      "linkedInUrl": "https://linkedin.com/in/bernt-omdal-073985133",
      "emails": [
        {
          "email": "bernt.omdal@sea1offshore.com",
          "validationStatus": "VALID"
        }
      ],
      "mobilePhones": [
        {
          "mobilePhone": "+4791522299",
          "confidenceScore": 0.75
        }
      ],
      "status": "COMPLETED"
    }
  ]
}
Enrichment data for ID 019734db-92a5-7021-9985-1f4793bd62f0:
{
  "enrichmentID": "019734db-92a5-7021-9985-1f4793bd62f0",
  "status": "COMPLETED",
  "percentCompleted": 100,
  "people": [
    {
      "externalID": "external-id",
      "firstName": "Vidar",
      "

In [76]:
key_contacts_enriched_raw

[{'enrichmentID': '019734db-6b2e-7c3d-aada-27169d88d7e6',
  'status': 'COMPLETED',
  'percentCompleted': 100,
  'people': [{'externalID': 'external-id',
    'firstName': 'Bernt',
    'lastName': 'Omdal',
    'companyName': 'Sea1 Offshore',
    'companyDomain': 'sea1offshore.com',
    'linkedInUrl': 'https://linkedin.com/in/bernt-omdal-073985133',
    'emails': [{'email': 'bernt.omdal@sea1offshore.com',
      'validationStatus': 'VALID'}],
    'mobilePhones': [{'mobilePhone': '+4791522299', 'confidenceScore': 0.75}],
    'status': 'COMPLETED'}]},
 {'enrichmentID': '019734db-92a5-7021-9985-1f4793bd62f0',
  'status': 'COMPLETED',
  'percentCompleted': 100,
  'people': [{'externalID': 'external-id',
    'firstName': 'Vidar',
    'lastName': 'Jerstad',
    'companyName': 'Sea1 Offshore',
    'companyDomain': 'sea1offshore.com',
    'linkedInUrl': 'https://linkedin.com/in/vidar-jerstad-2b81874',
    'jobTitle': 'Chief Financial Officer',
    'country': 'Norway',
    'emails': [{'email': 'vid

In [77]:
def extract_enriched_contact_details(enrichment_data):
    """
    Extract contact details from enrichment data into a standardized dictionary format
    
    Args:
        enrichment_data (dict): Raw enrichment data from Surfe API
        
    Returns:
        dict: Standardized contact details dictionary
    """
    contact_details = {}
    
    if enrichment_data.get('status') == 'COMPLETED' and enrichment_data.get('people'):
        person = enrichment_data['people'][0]
        
        # Extract name components
        first_name = person.get('firstName', '')
        last_name = person.get('lastName', '')
        full_name = f"{first_name} {last_name}".strip()
        
        # Extract email (get first valid email if exists)
        emails = person.get('emails', [])
        email = emails[0].get('email') if emails else None
        
        # Extract phone (get first mobile phone if exists)
        phones = person.get('mobilePhones', [])
        phone = phones[0].get('mobilePhone') if phones else None
        
        # Build contact dictionary
        contact_details = {
            "name": full_name,
            "role": person.get('jobTitle', ''),
            "LinkedIn_URL": person.get('linkedInUrl', ''),
            "phone": phone or '',  # Convert None to empty string
            "email": email or '',  # Convert None to empty string
            "country": person.get('country', ''),
            "company": person.get('companyName', ''),
            "domain": person.get('companyDomain', '')
        }
    
    return contact_details

key_contacts_enriched = []
# Process all enriched contacts
for enrichment in key_contacts_enriched_raw:
	key_contacts_enriched.append(extract_enriched_contact_details(enrichment))

print("Enriched Contacts:")
for contact in key_contacts_enriched:
	print(contact)

Enriched Contacts:
{'name': 'Bernt Omdal', 'role': '', 'LinkedIn_URL': 'https://linkedin.com/in/bernt-omdal-073985133', 'phone': '+4791522299', 'email': 'bernt.omdal@sea1offshore.com', 'country': '', 'company': 'Sea1 Offshore', 'domain': 'sea1offshore.com'}
{'name': 'Vidar Jerstad', 'role': 'Chief Financial Officer', 'LinkedIn_URL': 'https://linkedin.com/in/vidar-jerstad-2b81874', 'phone': '+4793426335', 'email': 'vidar.jerstad@sea1offshore.com', 'country': 'Norway', 'company': 'Sea1 Offshore', 'domain': 'sea1offshore.com'}
{'name': 'Tore Lillestø', 'role': 'Chief Operating Officer', 'LinkedIn_URL': 'https://linkedin.com/in/tore-lillestø-08b31b68', 'phone': '+47 990 21 180', 'email': 'tore.lillesto@sea1offshore.com', 'country': 'Norway', 'company': 'Sea1 Offshore', 'domain': 'sea1offshore.com'}
{'name': 'Andreas Kjøl', 'role': '', 'LinkedIn_URL': 'https://linkedin.com/in/andreas-kjøl-65678a37', 'phone': '+4795082287', 'email': 'andreas.kjol@sea1offshore.com', 'country': '', 'company': 

In [104]:
import requests
import json

# API Configurations
PIPEDRIVE_BASE_URL = "https://api.pipedrive.com/v1"
SURFE_BASE_URL = "https://api.surfe.io/v1"
SURFE_API_KEY = "ctksSOUV1-VRSY8i-FNgZW7IhyjnrDZOVYAsnQvzDOA"
person_id = 4693

# (Optional) Cache LinkedIn field key to avoid repeated lookups
LINKEDIN_FIELD_KEY = 'LinkedIn_URL'


def get_linkedin_data_from_surfe(email=None, domain=None, name=None, company=None):
    """Fetch LinkedIn URL using Surfe's API"""
    endpoint = f"{SURFE_BASE_URL}/profiles/find"
    headers = {"Authorization": f"Bearer {SURFE_API_KEY}"}
    payload = {}

    if email:
        payload["email"] = email
    if domain:
        payload["domain"] = domain
    if name and company:
        payload.update({"name": name, "company": company})

    response = requests.post(endpoint, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()
    return None

def create_or_update_pipedrive_person(email, name=None, phone=None, company=None):
    """Automatically enrich with Surfe and push to Pipedrive"""
    # Step 1: Get LinkedIn data from Surfe
    surfe_data = get_linkedin_data_from_surfe(email=email)
    if not surfe_data:
        print(f"No LinkedIn data found for {email}")
        return None

    linkedin_url = surfe_data.get("linkedin_url")
    if not linkedin_url:
        print("No LinkedIn URL found in Surfe response")
        return None

    # Step 2: Prepare Pipedrive payload
    person_data = {
        "name": name or surfe_data.get("full_name", "Unknown"),
        "email": email,
        "phone": phone or surfe_data.get("phone"),
        "org_id": get_or_create_company_id(company or surfe_data.get("company")),
        "linkedin_field_key": linkedin_url,
    }

    # Step 3: Check if person exists in Pipedrive
    existing_person = find_pipedrive_person(email)
    if existing_person:
        # Update existing person
        endpoint = f"{PIPEDRIVE_BASE_URL}/persons/{existing_person['id']}"
        method = "PUT"
    else:
        # Create new person
        endpoint = f"{PIPEDRIVE_BASE_URL}/persons"
        method = "POST"

    # Step 4: Push to Pipedrive
    params = {"api_token": PIPEDRIVE_API_TOKEN}
    response = requests.request(method, endpoint, json=person_data, params=params)
    
    if response.status_code in (200, 201):
        print(f"Success: {response.json()}")
        return response.json()
    else:
        print(f"Error: {response.text}")
        return None


def get_or_create_company_id(company_name):
    """Find or create organization in Pipedrive"""
    if not company_name:
        return None
    # (Implementation depends on your Pipedrive setup)
    # You can add similar logic as find_pipedrive_person() but for organizations
    return None  # Replace with actual org ID logic

# Example Usage
create_or_update_pipedrive_person(
    email="nils.liaaen@mokster.no",
    name="Nils Liaaen",  # Optional (Surfe can provide this)
    phone="+55 62 99615-1362",  # Optional
    company=""  # Optional
)

SSLError: HTTPSConnectionPool(host='api.surfe.io', port=443): Max retries exceeded with url: /v1/profiles/find (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1002)')))

In [79]:
owners_ids = {'Amine':22484151,'Prabhu':15645168,'Karthik':21576662,'Admin':22966424}

In [51]:
# Pipedrive integration
PIPEDRIVE_API_TOKEN = '7270c2d903c17226eef5e148fb214fba6c956a13'
owner_id = owners_ids['Karthik']
channel_id = 612

In [88]:
company_overview = json_data['company_overview']
key_contacts = json_data['key_contacts']
tailored_approach_strategy = json_data['tailored_approach_strategy']
general_principles= tailored_approach_strategy['general_principles']
contact_specific_outreach_methods = tailored_approach_strategy['contact-specific_outreach_methods']
sample_messaging = tailored_approach_strategy['sample_messaging']
rationale = tailored_approach_strategy['rationale']

In [89]:
def clean_contact_methods(methods):
    """
    Transform contact-specific outreach methods to use only names as keys
    
    Args:
        methods (dict): Original methods dictionary with "Name (Role)" format keys
        
    Returns:
        dict: Cleaned dictionary with only names as keys
    """
    cleaned_methods = {}
    
    for key, value in methods.items():
        # Check if there are multiple names separated by "/"
        if '/' in key:
            # Split the key into individual names
            names = [name.split('(')[0].strip() for name in key.split('/')]
            # Add the value for each name
            for name in names:
                cleaned_methods[name] = value
        else:
            # Handle single name case
            name = key.split('(')[0].strip()
            cleaned_methods[name] = value
    
    return cleaned_methods

# Transform the existing methods
contact_specific_outreach_methods = clean_contact_methods(contact_specific_outreach_methods)

In [86]:
organization = target_company
organization

'Sea1 Offshore'

In [85]:
def organization_exists(organization):
	# Check if organization exists
	words = organization.strip().split(' ')
	term = words[0] 
	print("Term",term)
	response = requests.get(
		'https://api.pipedrive.com/v1/organizations/find',
		params={'term': organization, 'api_token': PIPEDRIVE_API_TOKEN}
	)
	print(f"Checking if organization '{organization}' exists...",organization)
	print("Response",response)
	if response.status_code == 200:
		data = response.json()
		print("Data",data)
		if data['data']:
			print(f"Organization '{data['data'][0]['name']}' already exists")
			return data['data'][0]['id']  # Return the ID of the existing organization
		else:
			print(f"Organization '{organization}' does not exist")
			return None
	else:
		#print(f"Error checking organization: {response.status_code}")
		#print(response.text)
		return None

In [96]:
def create_organization_note_content(content):
    try:
        # Validate required keys exist
        required_keys = ['name', 'headquarters', 'core_business', 'size', 
                        'leadership', 'strategic_goals', 'overlay_opportunities']
        for key in required_keys:
            if key not in content:
                raise KeyError(f"Missing required key: {key}")

        # Create bullet points for lists ahead of time
        strategic_goals = "".join(f"• {goal}\n" for goal in content['strategic_goals'])
        opportunities = "".join(f"• {opp}\n" for opp in content['overlay_opportunities'])

        note_content = f"""# Company Overview: {content['name']}
        
📍 **Headquarters:**
{content['headquarters']}

🏢 **Core Business:**
{content['core_business']}

📊 **Company Size:**
{content['size']}

👥 **Leadership:**
{content['leadership']}

🎯 **Strategic Goals:**
{strategic_goals}
💡 **Opportunities Identified:**
{opportunities}"""

        return note_content.strip()  # Remove trailing whitespace

    except Exception as e:
        print(f"Error creating note content: {str(e)}")
        return None

In [97]:
import requests
# Search for the target company in Pipedrive
company_name = target_company

org_results = requests.get(
    'https://api.pipedrive.com/v1/organizations/search',  # Correct endpoint for org search
    params={
        'term': company_name,
        'api_token': PIPEDRIVE_API_TOKEN  # Pipedrive uses query param for auth in v1
    }
)

org_results = org_results.json()
if org_results.get('success', False):  # Safer way to check success
	if 'data' in org_results and 'items' in org_results['data']:
		if not org_results['data']['items']:
			print("No organizations found for the given term.")
			organization_id = None
		else:
			# Assuming you want the first match's ID
			organization_id = org_results['data']['items'][0]['item']['id'] if org_results['data']['items'] else None
			print(org_results)
			print("Organization ID:", org_results['data']['items'][0]['item']['id'])  # First match's ID
else:
    print("Organization not found or error occurred")
    if 'error' in org_results:
        print("Error:", org_results['error'])

{'success': True, 'data': {'items': [{'result_score': 0.35513002, 'item': {'id': 3857, 'type': 'organization', 'name': 'Sea1 Offshore', 'address': "Haakon VII's gate 2, Kristiansand, Norway", 'visible_to': 7, 'owner': {'id': 22966424}, 'custom_fields': [], 'notes': []}}]}, 'additional_data': {'pagination': {'start': 0, 'limit': 100, 'more_items_in_collection': False}}}
Organization ID: 3857


In [98]:
from datetime import datetime
# Get current date and time
current_time = datetime.now()

# Format current time as a string
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

In [94]:
import json
if organization_id:
	# Update Organization 
	response = requests.put(
		'https://api.pipedrive.com/v1/organizations',  # Changed to v1 from v2
		json=org_data,  # Use json parameter instead of params
		params={'api_token': PIPEDRIVE_API_TOKEN}  # API token in params
	)
	 # Extract the persons associated with the organization
	person_results = requests.get(
	   'https://api.pipedrive.com/api/v2/persons',
       params={
	  'api_token': PIPEDRIVE_API_TOKEN,
	  'org_id': organization_id}  # Pipedrive uses basic auth with API token
	)
	if person_results.status_code == 200:
		print("Persons found for organization ID:", organization_id)
		print(json.loads(person_results.text))
		person_data_json = json.loads(person_results.text)
		person_data = person_data_json.get('data', [])
		
	else:
		print("No persons found or error occurred")
else:
	#Check if organization id is None or not found
	# Create organization data
	org_data = {
		'name': target_company,
		'add_time': formatted_time,
		'05248db73ddbf1b366b6fc9eea1aba7dc29fc30e': country,  # Custom field for country
	}
	
	# Make POST request to create organization
	response = requests.post(
		'https://api.pipedrive.com/v1/organizations',  # Changed to v1 from v2
		json=org_data,  # Use json parameter instead of params
		params={'api_token': PIPEDRIVE_API_TOKEN}  # API token in params
	)
	
	# Check if request was successful
	if response.status_code == 201:  # 201 is success for POST
		print(f"Organization '{target_company}' created successfully")
		organization_id = response.json()['data']['id']
		print(f"Organization ID: {organization_id}")
		print(response.json())
	else:
		print(f"Error creating organization: {response.status_code}")
		print(response.text)

    



NameError: name 'organization_id' is not defined

In [62]:
# Create organization note content
note_content = create_organization_note_content(company_overview)
if note_content:
	# Create a note for the organization
	note_data = {
		'content': note_content,
		'org_id': organization_id,  # Use the correct key for organization ID
		'add_time': formatted_time,
	}

	# Make POST request to create note
	response = requests.post(
		'https://api.pipedrive.com/v1/notes',  # Correct endpoint for notes
		json=note_data,  # Use json parameter instead of params
		params={'api_token': PIPEDRIVE_API_TOKEN}  # API token in params
	)

	if response.status_code == 201:  # 201 is success for POST
		print("Note created successfully")
		print(response.json())
	else:
		print(f"Error creating note: {response.status_code}")
		print(response.text)

Note created successfully
{'success': True, 'data': {'id': 1076, 'user_id': 22966424, 'deal_id': None, 'person_id': None, 'org_id': 3867, 'lead_id': None, 'content': '# Company Overview: Simon Møkster Shipping<br />\n<br />\n📍 **Headquarters:**<br />\nStavanger, Norway<br />\n<br />\n🏢 **Core Business:**<br />\nSupplier of modern support vessels for the offshore energy sector (Rescue, NOFO Oil Recovery, Fire Fighting, Supply, Multi-field services) with a strong focus on harsh weather operations, digitalization, and sustainability.<br />\n<br />\n📊 **Company Size:**<br />\nFleet of 15 vessels; approximately 500 employees; main operational area: North Sea and Barents Sea<br />\n<br />\n👥 **Leadership:**<br />\nOwned by the Møkster family; Nils A. Liaaen (CCO/Technical Manager), Alf Mokster (CFO), Terje Gjerde (ICT Manager/CIO)<br />\n<br />\n🎯 **Strategic Goals:**<br />\n• Lead digital transformation in offshore support shipping<br />\n• Boost operational efficiency via modern fleet solu

In [99]:
import requests

# Configuration

PIPEDRIVE_BASE_URL = 'https://api.pipedrive.com/v1'

def update_linkedin_url(contact_id, linkedin_url):
    """
    Update a Pipedrive contact with a LinkedIn URL using your custom field
    
    :param contact_id: Pipedrive person ID
    :param linkedin_url: Full LinkedIn profile URL
    :return: API response
    """
    endpoint = f"{PIPEDRIVE_BASE_URL}/persons/{contact_id}"
    
    headers = {
        'Content-Type': 'application/json',
    }
    
    params = {
        'api_token': PIPEDRIVE_API_TOKEN
    }
    
    # The data structure uses your custom field key
    data = {
        'LinkedIn URL (Surfe)': linkedin_url
    }
    
    response = requests.put(endpoint, headers=headers, params=params, json=data)
    
    if response.status_code == 200:
        print(f"Successfully updated contact {contact_id} with LinkedIn URL: {linkedin_url}")
        return response.json()
    else:
        print(f"Error updating contact: {response.text}")
        return None



In [116]:
# Creat Contact person within the organization
def create_person(name, email, phone, organization_id, linkedin_url):
	required_keys = []
	person={}
	if name:
		required_keys.append('name')
		person['name']=name
	if email:
		required_keys.append('email')
		person['email']=email
	if phone:
		required_keys.append('phone')
		person['phone']=phone
	if organization_id:
		required_keys.append('org_id')
		person['org_id'] = organization_id
	if linkedin_url:
		required_keys.append('LinkedIn URL (Surfe)')
		person['f15405a0ba81d8a6d69467fa5048f52c6d6cb9a8'] = linkedin_url
	response = requests.post(
	   'https://api.pipedrive.com/v1/persons',
	   json=person,
	   params={'api_token': PIPEDRIVE_API_TOKEN}
	)

	print(response)
    
	if response.status_code in [201, 200, 202]:  # 201 is success for POST
		print(f"Person '{name}' created successfully")
		print(response.json()['data']['id'])
		return response.json()['data']['id']
		
	else:
		print(f"Error creating person: {response.status_code}")
		print(response.text)
		return None
     

In [117]:
# Create a note for each contact within the organization
def create_person_note_content(contact, general_principles, reformatted_methods, sample_messaging, rationale):
    # Create bullet points ahead of time
    principles_bullets = "".join(f"• {principle}\n" for principle in general_principles)
    rationale_bullets = "".join(f"• {r}\n" for r in rationale)
    
    # Create formatted note content with proper line breaks
    note_content = f"""# Contact Strategy: {contact['name']}

📋 **General Principles:**
{principles_bullets}

🎯 **Specific Outreach Method:**
{reformatted_methods.get(contact['name'], "No specific outreach method provided")}

💬 **Sample Messaging:**
{sample_messaging}

🔍 **Rationale:**
{rationale_bullets}"""

    return note_content.strip()

In [111]:
# Check if the contact for the organization already exists
def check_person_exists(name, organization_id):
	# Search for the person by name and organization ID
	search_results = requests.get(
	'https://api.pipedrive.com/v1/persons/search',
	params={
		'term': name,
		'org_id': organization_id,
		'api_token': PIPEDRIVE_API_TOKEN
	}
	)

	if search_results.status_code == 200:
		data = search_results.json()
		print(data)
		if data['success'] and data['data']['items']:
			return True, data['data']['items'][0]['item']['id']  # Return True and the person's ID
	return False, None  # Return False if not found

In [125]:
def update_person(person_id, name, email, phone, organization_id, linkedin_url):
	required_keys = []
	person={}
	if name:
		required_keys.append('name')
		person['name']=name
	if email:
		required_keys.append('email')
		person['email']=email
	if phone:
		required_keys.append('phone')
		person['phone']=phone
	if organization_id:
		required_keys.append('org_id')
		person['org_id'] = organization_id
	if linkedin_url:
		required_keys.append('LinkedIn URL (Surfe)')
		person['f15405a0ba81d8a6d69467fa5048f52c6d6cb9a8'] = linkedin_url
	
	
	response = requests.put(
		'https://api.pipedrive.com/v1/persons/{person_id}',
		json=person,
		params={'api_token': PIPEDRIVE_API_TOKEN}
	)
	print(response)
	return response.json()['data']['id'] if response.status_code == 200 else None

In [ ]:
import requests
result = update_person(
    person_id=4693,  # Replace with the actual person ID
    name="Nils Liaaen",
    phone= None,
    email= None,
    owner_id=22966424,
	organization_id=3867,
    linkedin_url="https://linkedin.com/in/nils-liaaen-055b7120",
    )
print(result)

TypeError: update_person() got an unexpected keyword argument 'owner_id'

In [126]:
organization_id

3857

In [122]:
key_contacts_enriched

[{'name': 'Bernt Omdal',
  'role': '',
  'LinkedIn_URL': 'https://linkedin.com/in/bernt-omdal-073985133',
  'phone': '+4791522299',
  'email': 'bernt.omdal@sea1offshore.com',
  'country': '',
  'company': 'Sea1 Offshore',
  'domain': 'sea1offshore.com'},
 {'name': 'Vidar Jerstad',
  'role': 'Chief Financial Officer',
  'LinkedIn_URL': 'https://linkedin.com/in/vidar-jerstad-2b81874',
  'phone': '+4793426335',
  'email': 'vidar.jerstad@sea1offshore.com',
  'country': 'Norway',
  'company': 'Sea1 Offshore',
  'domain': 'sea1offshore.com'},
 {'name': 'Tore Lillestø',
  'role': 'Chief Operating Officer',
  'LinkedIn_URL': 'https://linkedin.com/in/tore-lillestø-08b31b68',
  'phone': '+47 990 21 180',
  'email': 'tore.lillesto@sea1offshore.com',
  'country': 'Norway',
  'company': 'Sea1 Offshore',
  'domain': 'sea1offshore.com'},
 {'name': 'Andreas Kjøl',
  'role': '',
  'LinkedIn_URL': 'https://linkedin.com/in/andreas-kjøl-65678a37',
  'phone': '+4795082287',
  'email': 'andreas.kjol@sea1off

In [127]:
for contact in key_contacts_enriched:
	#id = contact['id']  # Assuming 'id' is the unique identifier for the contact
	name = contact.get('name', None)  # Using .get() with default value
	role = contact.get('role', None)
	linkedin_url = contact.get('LinkedIn_URL', None)  # Note the underscore in LinkedIn_URL
	phone = contact.get('phone', None)
	email = contact.get('email', None)


	# Extract contact id if contact exists otherwise create a new contact
	# Check if the contact already exists in Pipedrive
	

	contact_exists, person_id = check_person_exists(name, organization_id)
	if contact_exists:
	# Update existing person
		person_id = update_person(person_id, name, email, phone, organization_id, linkedin_url)
	else:
	# Create new person
		person_id = create_person(name, email, phone, organization_id, linkedin_url)
	

	person_id = int(person_id) if person_id else None  # Ensure person_id is an integer or None
	
	
		
	# Create a note for the person identified in the previous task
	note_data = {
	'content':  create_person_note_content(contact, general_principles, contact_specific_outreach_methods, sample_messaging, rationale),
	'person_id': person_id,  # Use the person_id returned from create_person,
	#'org_id': organization_id,  # Use the organization_id returned from create_organization
	'add_time': formatted_time
	}

	# Make POST request to create note
	response = requests.post(
	'https://api.pipedrive.com/v1/notes',
	json=note_data,
	params={'api_token': PIPEDRIVE_API_TOKEN}
	)

	# Check if request was successful
	if response.status_code == 201:
		print("Successfully added company overview note")
		print(response.json())
	else:
		print(f"Error adding note: {response.status_code}")
		print(response.text)

		
    # Now you can use org_id for further operations like creating deals or persons
	#print(f"Organization ID for {organization}: {org_id}")

{'success': True, 'data': {'items': [{'result_score': 0.61907, 'item': {'id': 4703, 'type': 'person', 'name': 'Bernt Omdal', 'phones': ['+4791522299'], 'emails': ['bernt.omdal@sea1offshore.com'], 'primary_email': 'bernt.omdal@sea1offshore.com', 'visible_to': 7, 'owner': {'id': 22966424}, 'organization': {'id': 3857, 'name': 'Sea1 Offshore', 'address': "Haakon VII's gate 2, Kristiansand, Norway"}, 'custom_fields': ['https://linkedin.com/in/bernt-omdal-073985133'], 'notes': ["# Contact Strategy: Bernt Omdal\n\n📋 **General Principles:**\n• Personalize communication for each executive by referencing Sea1 Offshores recent initiatives, growth, and ESG agenda.\n• Demonstrate deep understanding of Sea1's operational, technical, and compliance challenges in offshore shipping.\n• Emphasize Synergy’s digital, data-driven, and sustainability-driven ship management solutions (SMARTShip, decarbonization, global scale, compliance, cost efficiency).\n• Use a multi-channel approach: Connect via persona